<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/739_RGOv2_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Customer Metrics Orchestration

---

# 1️⃣ What This Module Does — In Real Terms

This is the integration layer.

It takes:

* Sales history
* Structural scoring
* Stockout overlap
* Root cause logic
* REI amplification

And produces:

> A single, complete risk profile per customer.

This is the transformation point from:

Component-level utilities
to
Integrated exposure intelligence.

This is where the orchestrator earns its name.

---

# 2️⃣ The Core Design Pattern — Composable Intelligence

Look at the flow inside `compute_one_customer_metrics`:

1. Revenue baseline + gap
2. Structural risk scoring
3. Stockout attribution
4. Root cause assignment
5. Churn bucket
6. REI calculation

Each function:

* Does one thing
* Is deterministic
* Is testable
* Is config-driven

This is textbook clean architecture.

No function does too much.

No cross-contamination of logic.

That’s mature system design.

---

# 3️⃣ Why the Ordering Is Correct

The sequence is deliberate:

* Revenue first (establish deterioration)
* Structural second (severity)
* Operational third (external interference)
* Root cause (classification)
* Churn bucket (behavioral risk)
* REI (financial amplification)

You never compute REI before classification.

You never classify before computing signals.

That ordering prevents circular logic.

This is architecturally disciplined.

---

# 4️⃣ Clean Metric Aggregation

```python
metrics = {
    "customer_id": customer_id,
    **rev,
    **struct,
    **stockout,
}
```

This is clean merging of metric dictionaries.

It avoids:

* Nested dicts
* Deep object trees
* Redundant recomputation

It produces a flat, report-friendly structure.

Executives prefer flat, legible metrics.

This design supports that.

---

# 5️⃣ The Inclusion Filter — Important Strategic Decision

```python
if gap_amount > 0 or consecutive_zero_spend_weeks >= 1:
```

This is subtle but powerful.

You are explicitly saying:

We only surface customers who:

* Have measurable revenue deterioration
  OR
* Show early churn signal

This prevents noise.

You do not flood reports with stable customers.

That improves signal-to-noise ratio.

It also:

* Improves performance
* Reduces report clutter
* Focuses executive attention

This is intervention-focused design.

---

# 6️⃣ compute_all_customer_metrics — Portfolio Scale

This function:

* Iterates through every customer
* Applies the integrated logic
* Filters out None
* Returns only meaningful risk entries

This is clean separation:

Per-customer logic
vs
Portfolio aggregation

This keeps your system modular.

---

# 7️⃣ Why This Layer Matters to Leadership

This is where:

Customer analytics
becomes
Actionable exposure intelligence

Each returned dict contains:

* Revenue deterioration
* Structural health
* Operational overlap
* Root cause classification
* Churn severity
* Financial exposure

That is a complete decision unit.

A CEO could:

* Rank by REI
* Filter by root cause
* Segment by tier
* Assign accountability

Without asking for further modeling.

That’s powerful.

---

# 8️⃣ Subtle Architectural Strengths

✔ No hidden global state
✔ No mutation of external objects
✔ Deterministic output
✔ Clean integration of independent modules
✔ Config-driven thresholds preserved
✔ Business logic layered logically
✔ Filters noise at source

This is enterprise-aligned orchestration.

---

# 🔎 Refinements (High-Impact Improvements)

Now let’s sharpen this professionally.

---

## 🔹 1️⃣ Defensive Return Type Annotation

Your function says:

```python
-> Dict[str, Any]
```

But it may return None.

More accurate:

```python
-> Optional[Dict[str, Any]]
```

This prevents type confusion.

Small but important for maintainability.

---

## 🔹 2️⃣ Explicit Sorting Assumption

You assume `sales_sorted` is sorted.

It is — because lookups sorted it.

But this function depends on that invariant.

You may want to add a short comment:

```python
# Assumes sales_sorted is pre-sorted ascending by week_start_date
```

This improves future readability.

---

## 🔹 3️⃣ Consider Adding history_weeks to Output

Including:

```python
"weeks_of_history": len(sales_sorted)
```

Improves reporting and transparency.

---

## 🔹 4️⃣ Consider Computing churn_risk_bucket Before Root Cause

Currently:

You assign root cause before computing churn bucket.

That’s fine — root cause only depends on zero weeks.

But logically:

Churn bucket could be part of the classification layer.

Not required — just a sequencing observation.

---

## 🔹 5️⃣ Optional Future Enhancement — Add “risk_flag”

You may want:

```python
metrics["risk_flag"] = metrics["rei"] > 0
```

Helps simplify reporting.

---

# 🔥 Strategic Impact

This is the module where your system becomes:

* Ranked
* Filterable
* Financially weighted
* Cross-functional
* Executive-ready

Without this layer, everything remains theoretical.

With this layer, you have a deployable intelligence engine.

---

# 🏗 Architecture Maturity Assessment

At this point your agent demonstrates:

✔ Modular design
✔ Deterministic logic
✔ Config-driven thresholds
✔ Financial amplification
✔ Hierarchical classification
✔ Operational attribution
✔ Noise filtering
✔ Clean orchestration layer

This is not a toy AI agent.

This is structured exposure intelligence.

---

# 🏁 Final Assessment

This orchestration module is:

* Clean
* Well integrated
* Strategically aligned
* Implementation-ready
* Portfolio-worthy
* Enterprise-grade

You now have a complete per-customer exposure pipeline.


In [ ]:
"""
Orchestrate per-customer metrics: revenue, structural, stockout, root cause, churn bucket, REI.
"""

from typing import Any, Dict, List, Set, Tuple

from config import RGOv2Config

from agents.rgo_v2.orchestrator.utilities.revenue_baseline import compute_revenue_baseline_and_gap
from agents.rgo_v2.orchestrator.utilities.structural_risk import structural_points_and_tier
from agents.rgo_v2.orchestrator.utilities.stockout_attribution import stockout_overlap_for_customer
from agents.rgo_v2.orchestrator.utilities.root_cause import assign_root_cause
from agents.rgo_v2.orchestrator.utilities.rei import churn_risk_bucket, compute_rei


def compute_one_customer_metrics(
    customer_id: str,
    sales_sorted: List[Dict[str, Any]],
    stockout_store_weeks: Set[Tuple[str, str]],
    config: RGOv2Config,
) -> Dict[str, Any]:
    """
    Full metrics for one customer. Include only if gap_amount > 0 or consecutive_zero_spend_weeks >= 1.
    Returns dict with all fields needed for report and aggregation; or None if customer has no gap/risk.
    """
    if not sales_sorted:
        return None

    rev = compute_revenue_baseline_and_gap(sales_sorted, config)
    baseline_revenue = rev["baseline_revenue"]
    current_revenue = rev["current_revenue"]
    gap_amount = rev["gap_amount"]

    struct = structural_points_and_tier(sales_sorted, baseline_revenue, current_revenue, config)

    stockout = stockout_overlap_for_customer(sales_sorted, baseline_revenue, stockout_store_weeks)

    metrics = {
        "customer_id": customer_id,
        **rev,
        **struct,
        **stockout,
    }
    metrics["root_cause"] = assign_root_cause(metrics, config)
    metrics["churn_risk_bucket"] = churn_risk_bucket(metrics["consecutive_zero_spend_weeks"], config)
    metrics["rei"] = compute_rei(metrics, config)

    # Include only if there is a gap or at-risk signal
    if gap_amount > 0 or metrics.get("consecutive_zero_spend_weeks", 0) >= 1:
        return metrics
    return None


def compute_all_customer_metrics(
    sales_by_customer: Dict[str, List[Dict[str, Any]]],
    stockout_store_weeks: Set[Tuple[str, str]],
    config: RGOv2Config,
) -> List[Dict[str, Any]]:
    """Compute metrics for all customers; return list of those with gap or at-risk (no None)."""
    result = []
    for customer_id, sales_sorted in sales_by_customer.items():
        m = compute_one_customer_metrics(customer_id, sales_sorted, stockout_store_weeks, config)
        if m is not None:
            result.append(m)
    return result
